![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate%2FConcepts&file=Compare+Documents.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Concepts/Compare%20Documents.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FConcepts%2FCompare%2520Documents.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Concepts/Compare%20Documents.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Concepts/Compare%20Documents.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Compare Documents

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [15]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.genai', 'google-genai'),
    ('pydantic', 'pydantic'),
    ('fitz', 'pymupdf'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'concept-doc-compare'

packages:

In [26]:
# standard packages
import os, shutil, enum, json, base64, typing, io

import IPython
import pydantic
import fitz

# google gen ai sdk
from google import genai

In [9]:
genai.__version__

'1.2.0'

clients:

In [10]:
genai_client = genai.Client(vertexai = True, project = PROJECT_ID, location = REGION)

---
## Multimodal Inputs

Prompts can extend beyond text to include audio, video, images, and documents as context.  This means you can [design multimodal prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/design-multimodal-prompts) that include combinations of these file types as grounding for Gemini.

### Get The Documents

A set of example file inputs for multimodal prompts were created in this companion workflow: [Create Files For Multimodal Prompt Examples](../Generate/Create%20Files%20For%20Multimodal%20Prompt%20Examples.ipynb) and the outputs are already included in the repository.

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [11]:
local_dir = '../files/multimodal-inputs'

In [12]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Generate/files/multimodal-inputs', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../files/multimodal-inputs`


---
### **Prompt With Documents**

[Document Understanding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/document-understanding)

#### Review The Documents

In [13]:
IPython.display.IFrame(f"{local_dir}/invoices.pdf", width=800, height= 800)

#### Read The Document Data

In [14]:
with open(f"{local_dir}/invoices.pdf", 'rb') as f:
    document_read = f.read()

#### Use PyMuPDF (fitz) To Extract Pages

In [17]:
document = fitz.open('pdf', document_read)

In [22]:
document.page_count

422

In [23]:
page_1 = fitz.open()
page_1.insert_pdf(document, from_page = 0, to_page = 0)

In [24]:
page_2 = fitz.open()
page_2.insert_pdf(document, from_page = 1, to_page = 1)

#### Pages As Byte Strings

In [27]:
buffer = io.BytesIO()
page_1.save(buffer)
page_1_bytes = base64.b64encode(buffer.getvalue()).decode('utf-8')

In [28]:
buffer = io.BytesIO()
page_2.save(buffer)
page_2_bytes = base64.b64encode(buffer.getvalue()).decode('utf-8')

#### Inline Loading

In [29]:
page_1_inline = genai.types.Part.from_bytes(
    data = page_1_bytes,
    mime_type = 'application/pdf'
)

In [30]:
page_2_inline = genai.types.Part.from_bytes(
    data = page_2_bytes,
    mime_type = 'application/pdf'
)

#### Generation Example

In [31]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [page_1_inline, 'Describe the content of this invoice.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0.3
    )
)

IPython.display.Markdown(response.text)

Here's a description of the invoice content:

**General Information:**

*   **Invoice Number:** 20231115-001
*   **Vendor:** Mike Demos
*   **Vendor Address:** 123 Tech Street, Silicon Valley, CA 94305
*   **Vendor Contact:** Phone: (555) 555-DEMO, Email: info@mikedemos.com
*   **Invoice Date:** November 15, 2023
*   **Due Date:** December 15, 2023

**Items/Services:**

The invoice lists the following items/services:

1.  **Plot Visualization (.png, .jpeg, .webp):** Quantity 2, Rate $160.00, Amount $320.00
2.  **Animated Plot Visualization (.mp4):** Quantity 1, Rate $220.00, Amount $220.00
3.  **Process Script & Audio (.mp3):** Quantity 3, Rate $110.00, Amount $330.00

**Financial Summary:**

*   **Subtotal:** $870.00
*   **Sales Tax (6%):** $52.20
*   **Total:** $922.20

**Payment Instructions:**

The invoice includes a note to remit payment to the address provided above. It also expresses gratitude for the business.

---
## Direct Compare (General)

In [32]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [page_1_inline, page_2_inline, 'Compare these two invoices highlighting any differences'],
    config = genai.types.GenerateContentConfig(
        temperature = 0.3
    )
)

IPython.display.Markdown(response.text)

Okay, let's compare the two invoices you provided and highlight the differences:

**Invoice #20231115-001 vs. Invoice #20240115-001**

**1. Invoice Number:**

*   Invoice #20231115-001
*   Invoice #20240115-001

**2. Address:**

*   Invoice #20231115-001: 123 Tech Street, Silicon Valley, CA 94305
*   Invoice #20240115-001: 123 Demo Street, Exampleville, CA 54321

**3. Phone Number:**

*   Invoice #20231115-001: (555) 555-DEMO
*   Invoice #20240115-001: (555) 555-1212

**4. Dates:**

*   Invoice #20231115-001:
    *   Invoice Date: November 15, 2023
    *   Due Date: December 15, 2023
*   Invoice #20240115-001:
    *   Invoice Date: January 15, 2024
    *   Due Date: February 14, 2024

**5. Line Items (Description, Quantity, Rate, Amount):**

*   Invoice #20231115-001:
    *   Plot Visualization (.png, .jpeg, .webp): Quantity 2, Rate $160.00, Amount $320.00
    *   Animated Plot Visualization (.mp4): Quantity 1, Rate $220.00, Amount $220.00
    *   Process Script & Audio (.mp3): Quantity 3, Rate $110.00, Amount $330.00
*   Invoice #20240115-001:
    *   Plot Visualization (.png, .jpeg, .webp): Quantity 2, Rate $160.00, Amount $320.00
    *   Animated Plot Visualization (.mp4): Quantity 1, Rate $220.00, Amount $220.00
    *   Interactive Data Dashboard: Quantity 1, Rate $400.00, Amount $400.00

**6. Subtotal:**

*   Invoice #20231115-001: $870.00
*   Invoice #20240115-001: $940.00

**7. Sales Tax:**

*   Invoice #20231115-001: 6%, $52.20
*   Invoice #20240115-001: 7%, $65.80

**8. Total:**

*   Invoice #20231115-001: $922.20
*   Invoice #20240115-001: $1005.80

**Summary of Key Differences:**

*   **Invoice Number:** Different, as expected.
*   **Address & Contact Info:** Different addresses and phone numbers.
*   **Dates:** Different invoice and due dates.
*   **Line Items:** Different third line item (Process Script & Audio vs. Interactive Data Dashboard), affecting the subtotal.
*   **Sales Tax:** Different tax rate (6% vs. 7%) and amount.
*   **Total Amount:** Different total due.

In essence, these are two distinct invoices for different services, dates, and locations, even though they share the same name.


---
## Controlled Responses With `response_schema`

### Targeted Extraction With Schema

In [33]:
class InvoiceData(pydantic.BaseModel):
    id: str = pydantic.Field(description = 'The unique invoice number or identifier.')
    company_name: typing.Optional[str] = pydantic.Field(description = 'The name of the company issuing the invoice.')
    line_item_total: typing.List[float] = pydantic.Field(description = 'A list of the total amount for each line item on the invoice.')
    total_amount: float = pydantic.Field(description = 'The overall total amount due on the invoice.')

In [34]:
response_page_1 = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [page_1_inline, 'Extract the requested information from this invoice.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0,
        system_instruction = 'Extract targeted information from invoices with high accuracy.',
        response_mime_type = 'application/json',
        response_schema = InvoiceData
    )
)

json.loads(response_page_1.text)

{'id': '20231115-001',
 'company_name': 'Mike Demos',
 'line_item_total': [320.0, 220.0, 330.0],
 'total_amount': 922.2}

In [35]:
response_page_2 = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [page_2_inline, 'Extract the requested information from this invoice.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0,
        system_instruction = 'Extract targeted information from invoices with high accuracy.',
        response_mime_type = 'application/json',
        response_schema = InvoiceData
    )
)

json.loads(response_page_2.text)

{'id': '20240115-001',
 'company_name': 'Mike Demos',
 'line_item_total': [320.0, 220.0, 400.0],
 'total_amount': 1005.8}

### Targeted Comparison With Schema

In [37]:
class ChangeSummary(pydantic.BaseModel):
    change: bool = pydantic.Field(description = 'Indicates whether a change occured between two documents.')
    summary: str = pydantic.Field(description = 'A text summary of changes, if any.')
    
class InvoiceChanges(pydantic.BaseModel):
    id: ChangeSummary = pydantic.Field(description = 'Change sumamry for the invoice ID.')
    company_name: ChangeSummary = pydantic.Field(description = 'Change sumamry for the company name.')
    line_item_total: ChangeSummary = pydantic.Field(description = 'Change sumamry for the list of line item totals.')
    total_amount: ChangeSummary = pydantic.Field(description = 'Change sumamry for the total amount.')

In [38]:
response_compare = genai_client.models.generate_content(
    model = 'gemini-2.0-flash-001',
    contents = [response_page_1.text, response_page_2.text, 'Compare these invoices.'],
    config = genai.types.GenerateContentConfig(
        temperature = 0,
        system_instruction = 'Extract targeted information from invoices with high accuracy and make comparisons to detect changes.',
        response_mime_type = 'application/json',
        response_schema = InvoiceChanges
    )
)

json.loads(response_compare.text)

{'id': {'change': True,
  'summary': 'The invoice ID changed from 20231115-001 to 20240115-001.'},
 'company_name': {'change': False,
  'summary': 'The company name remained the same: Mike Demos.'},
 'line_item_total': {'change': True,
  'summary': 'The line item totals changed. The third entry changed from 330.00 to 400.00.'},
 'total_amount': {'change': True,
  'summary': 'The total amount changed from 922.20 to 1005.80.'}}